In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px

df = pd.read_csv('dataset/ProcYelp.csv', index_col=0, low_memory=False)
df.head(5)

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,hours.Wednesday,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,Gi5LSRmTXoL9Bp4jNGPjLw,hn0ZbitvmlHnF--KJGJ6_A,TA1KUSCu8GkWP9w0rmElxw,4,0,0,0,I have been here twice and have had really goo...,2011-10-27 14:32:57,12:0-20:0,...,0,0,0,0,0,1,0,0,0,0
1,egHMQBhFeaLhVyC6w3r0vQ,HvpNr0ohHCaVLp014CQrdw,gGvNgShksetPoimyKV8I9Q,4,0,0,0,Quickly stopped in for a UFC fight. I sat down...,2015-10-04 05:37:01,11:0-2:0,...,0,0,0,0,0,0,0,0,0,0
2,GDgXjXSZCA1iNQWD7OHXfg,1RCRKuHgP3FskGUVnmFdxg,mOnesB4IF9j6-ZmHoOHOig,4,0,0,0,I think their rice dishes are way better than ...,2017-05-26 03:05:46,10:30-21:0,...,1,0,0,0,0,1,0,0,0,0
3,WxrK52MRJy6Cxk_vOJeD1Q,9d22a-5XXvsZ55p04gI5jA,_gOz7-aHMyGUHOtjDrEv2w,2,4,2,0,"When we first arrived at the Coco Key, we were...",2017-09-20 15:38:27,9:0-9:15,...,0,0,0,0,0,0,3,3,0,0
4,GgWFjRHhelaiUgR2-H6N4g,pRPT3vqhqpU7kHgmKJamvw,-_GnwXmzC3DXsHR9nyaC2g,3,0,0,0,3.5 stars! I got the avocado margarita and it ...,2012-11-06 07:09:57,11:0-21:0,...,0,0,0,0,0,1,0,0,0,0


## Zadání a očekávané otázky

### Zadání:
    Objednala si vás národní turistická agentura, abyste zhodnotili vybraná města a jejich návštěvníky podle recenzí podniků.
### Otázky
     Efa - Rozdělit recenze na pozitivní/negativní za použití SpaCY a pak dál řešit ty atributy města atd.?
        
        Jak se liší ->Podniky
                    ->Kategorie podniků dle města
                    ->Kategorie podniků a jak/jestli se liší jejich hodnocení
                    ->Podniky a jak/jestli se liší jejich hodnocení na základě různých vlastností (attributes, meals, ambience)
                    ->
                    ->Zákazníci
                    ->recenz zákazníků dle města (keywords, stars)
                    -> atd. nevím, seru na to dneska

In [38]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\aheldes\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [110]:
from nltk import tokenize
nltk.download('punkt')
def getScore(text):
    results = {}
    tokenized = tokenize.sent_tokenize(text)
    for sentence in tokenized:
        result_dict = sid.polarity_scores(sentence)
        result_dict.pop('compound', None)
        result_dict.pop('neu', None)
        max_key = max(result_dict, key=result_dict.get)
        if max_key in results:
            results[max_key] += 1
        else:
            results[max_key] = 1

    return max(results, key=results.get)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aheldes\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [111]:
df2 = df[:10000]
df2['score'] = df2.apply(lambda x: getScore(x.text), axis = 1)

C:\Users\aheldes\AppData\Local\Temp/ipykernel_25744/3861714374.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['score'] = df2.apply(lambda x: getScore(x.text), axis = 1)


In [115]:
df2["score"].value_counts()


pos    6513
neg    3487
Name: score, dtype: int64

In [116]:
df2["stars_x"].value_counts()

5    4097
4    2876
3    1343
2     863
1     821
Name: stars_x, dtype: int64

In [125]:
df3 = df2.loc[(df2['stars_x'] <= 3) & (df2['score'] != "neg")]

In [126]:
df3

,review_id,user_id,business_id,stars_x,useful_x,funny_x,cool_x,text,date,hours.Wednesday,...,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,score
4,GgWFjRHhelaiUgR2-H6N4g,pRPT3vqhqpU7kHgmKJamvw,-_GnwXmzC3DXsHR9nyaC2g,3,0,0,0,3.5 stars! I got the avocado margarita and it ...,2012-11-06 07:09:57,11:0-21:0,...,0,0,0,0,1,0,0,0,0,pos
11,EO5rALvJMkK8QEvUNs1gxg,u2xPfv6_wcKt-lW-C1cV8A,9P-lp3AWDXGayDqJz9VPwQ,2,0,0,0,The ramen here is less than great. It came out...,2018-02-11 03:30:12,11:0-20:0,...,0,0,0,0,0,0,0,0,0,pos
25,PWKCb5AoQDfkP0va6pLwWQ,YxvuZGAYcJIEQJmgtRCi3g,LGZimmjxbfGwBHgPZCx5cQ,2,2,1,0,So I tried calling multiple times because the ...,2017-07-18 18:21:42,11:0-22:0,...,0,0,0,12,5,11,11,16,4,pos
31,mEr_SpcpRzaajigrj0W6KA,TbeNFuahhKF3C5oJRT_ukQ,YZs1gNSh_sN8JmN_nrpxeA,3,1,0,0,Went here on Sunday for first time. Plenty of ...,2014-08-10 19:48:18,11:0-21:0,...,1,0,0,1,2,2,2,2,0,pos
37,PITWfZs-zmNMnHBXNLpLPw,4KgTmD0CULuQsrHouZ6zTA,OlB0841vj4V7Wje5tTtLWA,3,0,0,0,What a quaint little authentic Argentinan rest...,2010-07-07 20:51:00,12:0-21:0,...,1,9,3,25,30,44,44,23,10,pos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9981,yDPMS_9fzsfyQMIP2U68hQ,JyaXWjzIyU1anjmY2bsQ1Q,I-fCZYwlO0eZDu9OhDVG_g,3,1,0,0,A lot of fun! Beware the front row will get yo...,2014-11-30 23:19:52,8:0-20:0,...,4,1,0,10,11,5,5,3,1,pos
9982,O2diWGUUbCMHd8lFQOX5Vw,a8ooLTqQmxPuosJWCSzYzQ,ZHA_Wk3D9lDMR1bd7QT-9A,1,0,0,0,Went here for happy hour with coworkers. Food ...,2018-07-26 23:39:43,16:0-21:0,...,0,0,0,0,0,0,0,0,0,pos
9987,poWLcpYYV02iF622nUMLqw,dQeOk5L1HaX3vjQrLifs2Q,TA1KUSCu8GkWP9w0rmElxw,2,0,3,0,Great until the end.... Waiting like 20 mins f...,2016-01-30 23:25:57,12:0-20:0,...,0,0,0,3,7,4,4,0,0,pos
9988,-UVnFLh2ioBC5shAttAR-Q,qtxaDsVN2ci6g5v8cQ3lpA,IdXHHEUH4ebcxdRxCo3JNw,3,0,0,0,Very nice neighborhood restaurant right across...,2013-04-13 02:34:53,17:0-22:0,...,0,0,0,2,0,2,2,6,0,pos
